# METODOS NUMERICOS Y PROGRAMACION 4
## Tarea 3
##### Daniel Valencia - Computacion Cientifica - 2018-1
---

# Solucion numerica de EDPs
---

<ol>
    <li> Ecuaciones Parabolicas 
        <ul>
            <li> Metodo Explicito </li>
            <li> Metodo Implicitod </li>
            <li> Metodo Crank - Nicolson </li>
        </ul>
    </li>
    <li> Ecuaciones Hiperbolicas
        <ul>
            <li> Ecuacion de la onda </li>
        </ul>
    </li>
    <li> Ecuaciones Elipticas 
        <ul>
            <li> Ecuacion de Poisson </li>
            <li> Ecuacion de Laplace </li>
        </ul>
    </li>    
</ol>


## 1) EDP Parabolicas
---

Se va a considerar la ecuacion diferencial parcial del calor o de difusion:
$$\frac{\partial U}{\partial t} (x,t) = \alpha ^2 \frac{\partial ^2 U}{\partial x^2} (x,t) \text{  ,  } 0 < x < l \text{  , } t > 0$$
Sujeta a las condiciones: 
$$U(0,t) = U(l,t) = 0 \text{  ,  } t>0 \text{  y  }  U(x,0) = f(x) \text{  ,  } 0 < x < l$$

### - Metodo Explicito
---
Tomando: $$U(x_i, t_j) = w_{ij}$$
Se obtiene la forma matricial: 
$$
(1-2\lambda)w_{ij} + \lambda w_{i+1,j} + \lambda w_{i-1,j} = w_{i,j-1} \text{ , con } w_{i0} = f(x_i)
$$
$$
\begin{bmatrix}
    (1-2\lambda) & \lambda & & \dots & 0 \\
    \lambda & (1-2\lambda) & \lambda & \dots & 0\\
    0 & \lambda & & & \vdots\\
    \vdots& & \ddots &  & 0\\
     & & & & \lambda \\
    0 & & & \lambda & (1-2\lambda)
\end{bmatrix} = 
\begin{bmatrix}
    w_{1j} \\
    w_{2j} \\
    w_{3j} \\
    \vdots \\
           \\
    w_{m-1,j}
\end{bmatrix} = 
\begin{bmatrix}
    w_{1,j-1} \\
    w_{2,j-1} \\
    w_{3,j-1} \\
    \vdots \\
           \\
    w_{m-1,j-1}
\end{bmatrix}
$$
Con $\displaystyle \lambda = \frac{\alpha ^2 k}{h^2}$

#### Ejemplo (Burden - pag 727)
Vamos a resolver el siguiente ejempo: 

Usando los tamanos de paso $h = 0.1$ y $k = 0.0005$ para aproximar la solucion a la ecuacion de calor: 
$$\frac{\partial U}{\partial t} (x,t)  - \frac{\partial ^2 U}{\partial x^2} (x,t)  = 0 \text{  ,  } 0 < x < 1 \text{  ,  } 0 \leq t$$
Con las condiciones frontera: 
$$U(0,t) = U(l,t) = 0 \text{  ,  } 0 < t$$
Y las condiciones iniciales: 
$$U(x,0) = sin(\pi x) \text{  ,  } 0 \leq x \leq 1$$

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')

#### A partir de ahora se usaran las siguientes variables para los metodos: 

<ul>
    <li> c: Difusividad termica.</li>
    <li> L: Longitud de la barra.</li>
    <li> T: Tiempo final.</li>
    <li> h: Step-size que divide a la barra. </li>
    <li> k: Step-size que divide al tiempo.</li>
    <li> f: Condicion inicial U(x,0) = f(x).</li>
    <li> a: Condicion inicial de frontera en U(0,t).</li>
    <li> b: Condicion inicial de frontera en U(l,t).</li>
</ul>

In [ ]:
def tridiag(a,b,c,N):
    A = np.zeros((N,N))
    
    np.fill_diagonal(A[:-1,1:],a)
    np.fill_diagonal(A,b)
    np.fill_diagonal(A[1:,:-1],c)
    
    return A

#### Solucion exacta: 

In [ ]:
def Exacta(x,t):
    n = len(x)
    sol = []
    for i in range(n):
        sol.append((np.exp((-1)*(np.pi**2)*(t)))*np.sin(np.pi*x[i]))
    return sol

#### Error absoluto

In [ ]:
def ErrorAb(a,b):
    n = len(a)
    c = []
    for i in range(n):
        c.append(np.absolute(a[i] - b[i]))
    return c

### - Implementacion en Python

In [ ]:
def EDP_Exp_Matricial(c, L, T, h, k, f, a, b):
    
    r = c*k/h**2
    m = round(L/h) + 1
    n = round(T/k) + 1
    
    x = np.linspace(0, L, m)
    
    w = np.zeros((n,m))
    w[0] = f(x)
    w[:,0] = a(0) 
    w[:,-1] = b(L)
    
    W = tridiag(r, 1-2*r, r, m-2) 
    
    for i in range(n-1):
        w[i+1,1:-1] = np.dot(W, w[i,1:-1])
        w[i+1,1] += r*w[i,0]
        w[i+1,-2] += r*w[i,-1]
        
    return w,x

##### Los datos para el ejemplo: 

In [ ]:
Mat, x1 = EDP_Exp_Matricial(1, 1, 0.5, 0.1, 0.0005, lambda x: np.sin(np.pi * x), lambda x: 0, lambda x:0)

Exa1 = Exacta(x1,0.5)
Err1 = ErrorAb(Mat[-1],Exa1)
d = {'x': x1 , 'Aproximacion': Mat[-1], 'Exacta': Exa1, 'Error': Err1}
df = pd.DataFrame(data=d)
df = df[['x','Aproximacion','Exacta','Error']]
df

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

x = np.linspace(0, 1, 11)
t = np.linspace(0, 1, 1001)

def init():
  ax.plot([], [])
  ax.grid()
  ax.set_ylim(0, 1)
  ax.set_ylabel("Distribucion en el eje Y ")
  ax.set_xlabel("Posición en la cuerda")
  ax.set_title("Longitud de la barra ")

def animate(i):
  del ax.lines[:]
  ax.plot(x, Mat[i], color="blue", label="aprox. $t = {0:.2f}$ seg.".format(t[i]))
  ax.legend()
  
animation.FuncAnimation(fig, animate, init_func=init, frames=1001, interval=200, repeat=True)

### - Metodo Implicito
---
Se onsidera la siguiente forma matricial:
$$
(1+2\lambda)w_{ij} - \lambda w_{i+1,j} - \lambda w_{i-1,j} = w_{i,j-1} \text{ , con } w_{i0} = f(x_i)
$$
$$
\begin{bmatrix}
    (1+2\lambda) & -\lambda & & \dots & 0 \\
    - \lambda & (1+2\lambda) & -\lambda & \dots & 0\\
    0 & -\lambda & & & \vdots \\
    \vdots & & \ddots &  & 0\\
     & & & & -\lambda \\
    0 & & & -\lambda &(1+2\lambda)
\end{bmatrix} = 
\begin{bmatrix}
    w_{1j} \\
    w_{2j} \\
    w_{3j} \\
    \vdots \\
           \\
    w_{m-1,j}
\end{bmatrix} = 
\begin{bmatrix}
    w_{1,j-1} \\
    w_{2,j-1} \\
    w_{3,j-1} \\
    \vdots \\
           \\
    w_{m-1,j-1}
\end{bmatrix}
$$
#### - Ejemplo (Burden - pag 727)
Vamos a resolver el mismo ejempo que en el caso Explicito: 

Usando los tamanos de paso $h = 0.1$ y $k = 0.0005$ para aproximar la solucion a la ecuacion de calor: 
$$\frac{\partial U}{\partial t} (x,t)  - \frac{\partial ^2 U}{\partial x^2} (x,t)  = 0 \text{  ,  } 0 < x < 1 \text{  ,  } 0 \leq t$$
Con las condiciones frontera: 
$$U(0,t) = U(l,t) = 0 \text{  ,  } 0 < t$$
Y las condiciones iniciales: 
$$U(x,0) = sin(\pi x) \text{  ,  } 0 \leq x \leq 1$$

### - Implementacion en Python

In [ ]:
def EDP_Imp_Matricial(c, L, T, h, k, f, a, b):
    
    r = c*k/h**2
    m = round(L/h) + 1
    n = round(T/k) + 1
    
    x = np.linspace(0, L, m)
    
    w = np.zeros((n,m))
    w[0] = f(x)
    w[:,0] = a(0) 
    w[:,-1] = b(L)
    
    W = tridiag(-r, 1+2*r, -r, m-2) 

    W_inv = np.linalg.inv(W)
    
    for i in range(n-1):
        B = w[i,1:-1].copy() 
        B[0] += r*w[i,0]
        B[-1] += r*w[i,-1]
        w[i+1,1:-1] = np.dot(W_inv, B)
        
    return w,x

#### Los datos para el ejemplo: 

In [ ]:
Mat2, x2 = EDP_Imp_Matricial(1, 1, 0.5, 0.1, 0.0005, lambda x: np.sin(np.pi * x), lambda x: 0, lambda x:0)
Exa2 = Exacta(x2,0.5)
Err2 = ErrorAb(Mat2[-1],Exa2)

d2 = {'x': x2 , 'Aproximacion': Mat2[-1], 'Exacta': Exa2, 'Error': Err2}
df2 = pd.DataFrame(data=d2)
df2 = df2[['x','Aproximacion','Exacta','Error']]
df2

In [ ]:
fig2 = plt.figure()
ax2 = fig2.add_subplot(1,1,1)

x2 = np.linspace(0, 1, 11)
t2 = np.linspace(0, 1, 1001)

def init2():
  ax2.plot([], [])
  ax2.grid()
  ax2.set_ylim(0, 1)
  ax2.set_ylabel("Distribucion en el eje Y ")
  ax2.set_xlabel("Posición en la cuerda")
  ax2.set_title("Longitud de la barra ")

def animate2(i):
  del ax2.lines[:]
  ax2.plot(x2, Mat2[i], color="blue", label="aprox. $t = {0:.2f}$ seg.".format(t[i]))
  ax2.legend()
  
animation.FuncAnimation(fig2, animate2, init_func=init2, frames=1001, interval=200, repeat=True)

### - Metodo Crank - Nicolson
---
Se onsidera la siguiente forma matricial:
$$
\frac{w_{i,j+1} - w_{ij}}{k} - \frac{\alpha^2}{2} \left[ \frac{w_{i+1,j} - 2w_{ij} + w_{i-1,j}}{h^2} + \frac{w_{i+1,j+1} - 2w_{i,j+1} + w_{i-1,j+1}}{h^2} \right ] = 0
$$
$$ A w^{(j+1)}  \text{  =  } B w^{(j)} $$
$$
A = \begin{bmatrix}
    (1+\lambda) & -\frac{\lambda}{2} & & \dots & 0 \\
    -\frac{\lambda}{2} & (1+\lambda) & -\frac{\lambda}{2} & \dots & 0\\
    0 & -\frac{\lambda}{2} & & & \vdots \\
    \vdots & & \ddots &  & 0\\
     & & & & -\frac{\lambda}{2} \\
    0 & & & -\frac{\lambda}{2} & (1+\lambda) 
    \end{bmatrix}
\text{  ,  } 
B = \begin{bmatrix}
    (1-\lambda) & \frac{\lambda}{2} & & \dots & 0 \\
    \frac{\lambda}{2} & (1-\lambda) & \frac{\lambda}{2} & \dots & 0\\
    0 & \frac{\lambda}{2} & & & \vdots \\
    \vdots & & \ddots &  & 0\\
     & & & & \frac{\lambda}{2} \\
    0 & & & \frac{\lambda}{2} & (1-\lambda) 
    \end{bmatrix}
$$

#### - Ejemplo (Burden- pag 727)
Vamos a resolver el mismo ejempo que en el caso Explicito: 

Usando los tamanos de paso $h = 0.1$ y $k = 0.0005$ para aproximar la solucion a la ecuacion de calor: 
$$\frac{\partial U}{\partial t} (x,t)  - \frac{\partial ^2 U}{\partial x^2} (x,t)  = 0 \text{  ,  } 0 < x < 1 \text{  ,  } 0 \leq t$$
Con las condiciones frontera: 
$$U(0,t) = U(l,t) = 0 \text{  ,  } 0 < t$$
Y las condiciones iniciales: 
$$U(x,0) = sin(\pi x) \text{  ,  } 0 \leq x \leq 1$$

### - Implementacion en Python

In [ ]:
def EDP_CN_Matricial(c, L, T, h, k, f, a, b):
    
    r = c*k/h**2
    m = round(L/h) + 1
    n = round(T/k) + 1
    
    x = np.linspace(0, L, m)
    
    w = np.zeros((n,m))
    w[0] = f(x)
    w[:,0] = a(0) 
    w[:,-1] = b(L)
    
    A = tridiag(-r/2, 1+r, -r/2, m-2)
    B = tridiag(r/2, 1-r, r/2, m-2)

         
    A_inv = np.linalg.inv(A)


    for i in range(n-1):
        
        C = np.dot(B, w[i,1:-1])
        C[0]  += r/2*(w[i+1,0]  + w[i,0])
        C[-1] += r/2*(w[i+1,-1] + w[i,-1])

        w[i+1,1:-1] = np.dot(A_inv, C)
    
    return w, x

#### Los datos para el ejemplo: 

In [ ]:
Mat3, x3 = EDP_CN_Matricial(1, 1, 0.5, 0.1, 0.0005, lambda x: np.sin(np.pi * x), lambda x: 0, lambda x:0)
Exa3 = Exacta(x3,0.5)
Err3 = ErrorAb(Mat3[-1],Exa3)
d3 = {'x': x3 , 'Aproximacion': Mat3[-1], 'Exacta': Exa3, 'Error': Err3}
df3 = pd.DataFrame(data=d3)
df3 = df3[['x','Aproximacion','Exacta','Error']]
df3

In [ ]:
fig3 = plt.figure()
ax3 = fig3.add_subplot(1,1,1)

x3 = np.linspace(0, 1, 11)
t3 = np.linspace(0, 1, 1001)

def init3():
  ax3.plot([], [])
  ax3.grid()
  ax3.set_ylim(0, 1)
  ax3.set_ylabel("Distribucion en el eje Y ")
  ax3.set_xlabel("Posición en la cuerda")
  ax3.set_title("Longitud de la barra ")

def animate3(i):
  del ax3.lines[:]
  ax3.plot(x3, Mat3[i], color="blue", label="aprox. $t = {0:.2f}$ seg.".format(t[i]))
  ax3.legend()
  
animation.FuncAnimation(fig3, animate3, init_func=init3, frames=1001, interval=120, repeat=True)

## 2) EDP Hiperbolicas
---

Se va a considerar la ecuacion diferencial parcial de la onda, dada por la ecuacion diferencial:
$$\frac{\partial^2 U}{\partial^2 t} (x,t) - \alpha ^2 \frac{\partial ^2 U}{\partial x^2} (x,t) = 0 \text{  ,  } 0 < x < l \text{  , } t > 0$$
Sujeta a las condiciones: 
$$U(0,t) = U(l,t) = 0 \text{  ,  } t>0 $$
$$ U(x,0) = f(x) \text{  y  } \frac{\partial U}{\partial t} (x,t) = g(x) \text{ , para } 0 < x < l$$

Se obtendra la solucion numerica a partir de: 
$$ w_{i,j+1} = 2(1-\lambda^2) w_{ij} + \lambda^2(w_{i+1,j} + w_{i-1,j}) - w_{i,j-1} \text{  con  } \lambda = \frac{\alpha k}{h}$$
Y, expresado en su forma matricial, se tiene:
$$
\begin{bmatrix}
    w_{1,j+1} \\
    w_{2,j+1} \\
    w_{3,j+1} \\
    \vdots \\
           \\
    w_{m-1,j+1}
\end{bmatrix}
= \begin{bmatrix}
    (2-\lambda^2) & \lambda^2 & & \dots & 0 \\
    \lambda^2 & (2-\lambda^2) & \lambda^2 & \dots & 0\\
    0 & \lambda^2 & & & \vdots \\
    \vdots & & \ddots &  & 0\\
     & & & & \lambda^2 \\
    0 & & & \lambda^2 & (2-\lambda^2) 
    \end{bmatrix}
\begin{bmatrix}
    w_{1j} \\
    w_{2j} \\
    w_{3j} \\
    \vdots \\
           \\
    w_{m-1,j}
\end{bmatrix}
-
\begin{bmatrix}
    w_{1,j-1} \\
    w_{2,j-1} \\
    w_{3,j-1} \\
    \vdots \\
           \\
    w_{m-1,j-1}
\end{bmatrix}
$$

En la siguiente implementacion se tiene: 
$$f = U(x,0) \text{  ,  } g = U_t (x,0) \text{ : Son las condiciones iniciales.  }$$
$$a = U(0,t) \text{  ,  } b = U(l,t) \text{ : Son las condiciones de frontera. } $$

### - Implementacion en Python

In [ ]:
def EDP_Hiperbolica_Mat(c, L, T, h, k, f, g, a, b):
    #f = U(x,0) , g = U_t(x,0) (Condiciones iniciales)
    #a = U(0,t) , b = U(l,t) ( Condiciones frontera)
    
    r = (c*k**2)/h**2
    n = round(T/k) + 1
    m = round(L/h) + 1

    x = np.linspace(0,L,m)
    w = np.zeros((n,m))
    w[0] = f(x)
   
    for j in range(1,m-1):
        w[1,j] = (1-r)*f(x[j]) + (r/2)*f(x[j+1]) + (r/2)*f(x[j-1]) + k*g(x[j])

    w[:,0] = a(0)
    w[:,-1] = b(L)

    W = tridiag(r, 2-2*r, r, m-2)

    for i in range(1,n-1):
        w[i+1,1:-1] = np.dot(W, w[i,1:-1]) - w[i-1,1:-1]
        w[i+1,1] += r*w[i,0]
        w[i+1,-2] += r*w[i,-1]
        
    return w, x

Usando los tamanos de paso h=0.1 y k=0.05 para aproximar la solucion al problema hiperbolico:
$$\frac{\partial ^2U}{\partial t^2}(x,t) - 4\frac{\partial ^2U}{\partial x^2}(x,t) = 0 \text{  ,  } 0<x<l \text{  ,  } 0 < t.$$
Con las condiciones frontera: $$U(0,t) = U(1,t) = 0 \text{ , para } 0 < t$$
Y las condiciones iniciales: $$U(x,0)  = sin (\pi x) \text{  ,  } 0 \leq x \leq 1 \text{  , y  } \frac{\partial U}{\partial t} (x,0)  = 0 \text{  ,  } 0\leq x \leq 1$$
#### Los datos para el ejemplo: 

In [ ]:
def Exacta2(x,t):
    n = len(x)
    sol = []
    for i in range(n):
        sol.append((np.sin(np.pi * x[i])) * (np.cos(2*np.pi * t)))
    return sol
    
HipH, xH = EDP_Hiperbolica_Mat(4, 1, 1, 0.1, 0.05, lambda x: np.sin(np.pi * x), lambda x: 0, lambda x: 0, lambda x:0)
ExaH = Exacta2(xH,0.5)
ErrH = ErrorAb(HipH[-1],ExaH)
dH = {'x': xH , 'Aproximacion': HipH[-1], 'Exacta': ExaH, 'Error': ErrH}
dH = pd.DataFrame(data=dH)
dH = dH[['x','Aproximacion','Exacta','Error']]
dH

In [ ]:
figH = plt.figure()
axH = figH.add_subplot(1,1,1)

xH = np.linspace(0, 1, 11)
tH = np.linspace(0, 1, 21)

def initH():
  axH.plot([], [])
  axH.grid()
  axH.set_ylim(-1, 1)
  axH.set_ylabel("ALTURA")
  axH.set_xlabel("Posición en la cuerda")
  axH.set_title("Movimiento de la cuerda")

def animateH(i):
  del axH.lines[:]
  axH.plot(xH, HipH[i], color="blue", label="aprox. $t = {0:.2f}$ seg.".format(tH[i]))
  axH.legend()
  
animation.FuncAnimation(figH, animateH, init_func=initH, frames=21, interval=120, repeat=True)

## 3) EDP Elipticas
---

Se va a considerar la ecuacion diferencial parcial de Poisson, dada por la ecuacion diferencial:
$$\nabla ^2 U(x,y) = \frac{\partial^2 U}{\partial^2 x} (x,y) + \frac{\partial ^2 U}{\partial y^2} (x,y) = f(x,y)$$
$ \text{ Sobre  } R= \{ (x,y) \text{  |  } a < x < b , c < y < d\} $

$\text{ Con  } U(x,y) = g(x,y), \text{  para  } (x,y) \in S \text{, donde S denota la frontera de R.} $

### - Ecuacion de Laplace: 
---
Por otro lado, el estudio de distribuciones de calor estables en una region plana requiere que en la ecuacion de Poisson se tome $f(x,y) = 0$, resulta en una simplificacion llamada Ecuacion de Laplace: $$\nabla ^2 U(x,y) = \frac{\partial^2 U}{\partial^2 x} (x,y) + \frac{\partial ^2 U}{\partial y^2} (x,y) = 0$$

#### - Metodo de diferencias finitas: 
En la forma de ecuaciones en diferencias, se obtiene el metodo de diferencias finitas: 
$$2\left[ \left( \frac{h}{k} \right)^2 + 1 \right] w_{ij} - (w_{i+1,j} + w_{i-1,j}) - \left(\frac{h}{k}\right)(w_{i,j+1} + w_{i,j-1}) = h^2 f(x_i,y_i)$$

Para cada $i = 1,2 \dots n-1$ y $ j = 1, 2, \dots m-1$, y adem\'as: 
$$w_{0j} = g(x_0,y_j) \text{  -  } w_{nj} = g(x_n,y_j) \text{  para cada  } j = 0, 1, \dots m$$
$$w_{i0} = g(x_i,y_0) \text{  -  } w_{im} = g(x_i,y_m) \text{  para cada  } i = 1, 2, \dots m-1$$

Donde los $w_{ij}$ aproximan los $U(x_i, y_j)$

Se obtiene una matriz tridiagonal de la forma: 

$$
\begin{bmatrix}
M & E & 0 &  & & \dots &0 \\
E & M & E & 0 &  &\dots & 0\\
0 & E & M & E & 0 & \dots & 0 \\
  &    & \ddots & \ddots & \ddots & \dots & 0 \\
 & & & & E & M & E\\
 & & & & 0 & E & M
\end{bmatrix}
$$

Donde: 

$ M = 
\begin{bmatrix}
-2(1+\lambda) & 1 & & & & \dots & 0 \\
 1 & -2(1+\lambda) & 1 & & & \dots & 0 \\
0 & 1 & -2(1+\lambda) & 1 & & \dots & 0 \\
  & & \ddots & \ddots & & \dots & 0 \\
  & & & & &  & 1 \\
  & & & & &  1 & -2(1+\lambda)
\end{bmatrix}
$
,
$
E = 
\begin{bmatrix}
\lambda & & & & & & \\
& \lambda & & & & & \\
& & \lambda & & & & \\
& & & \ddots & & & \\
& & & & & \lambda & \\
& & & & & &\lambda \\
\end{bmatrix}
$

#### - Ejemplo 1 (Burden - pag 718)
Vamos a resolver un ejemplo para la ecuacion de Laplace: 

$$\frac{\partial^2 U}{\partial^2 x} (x,y) + \frac{\partial ^2 U}{\partial y^2} (x,y) = 0$$
$ \text{ Sobre  } R= \{ (x,y) \text{  |  } 0 < x < 0.5 , 0 < y < 0.5\} $

Sujeta a las condiciones frontera:

$$U(0,y) = 0, U(x,0) = 0 , U(x,0.5) = 200x , U(0.5,y) = 200y$$

### - Implementacion en Python

$$f(x, y) \text{ : Funcion (0 : Laplace, $\neq$ 0 : Poisson)} $$
$$L_l \text{ : Limite izquierdo} $$
$$L_r \text{ : Limite derecho} $$
$$L_i \text{ : Limite inferior} $$
$$L_u \text{ : Limite superior} $$

In [ ]:

def EDP_Eliptica_Mat(a, b, c, d, h, k, f, L_l, L_r, L_i, L_u):
    
    r = (h**2)/k**2

    n = round((b-a)/h) + 1
    m = round((d-c)/k) + 1

    x = np.linspace(a , b, n);
    y = np.linspace(c , d, m);

    p = (n-2)*(m-2)
     
    C = tridiag(r, -2*(1+r), r, n-2)

    A = np.kron(np.eye(m-2), C)
    
    np.fill_diagonal(A[:(p-(n-2)),(n-2):],r)

    np.fill_diagonal(A[(n-2):,:(p-(n-2))],r)

    xx,yy = np.meshgrid(x[1:-1],y[1:-1])
    
    B = np.zeros((m-2,n-2))
    
    for i in range(m-2):
        for j in range(n-2):
            B[i,j] = (h**2) * f(x[j+1], y[-(i+2)]) 

    for i in range(m-2):  
        B[i,0] -= L_l( y[-(i+2)] )
        B[i,-1] -= L_r( y[-(i+2)] )         

    for j in range(n-2):
        B[0,j] -= r*L_u(x[j+1])
        B[-1,j] -= r*L_i(x[j+1]) 
    
    B = np.reshape(B, np.size(B))

    W = np.linalg.solve(A, B)
    W1 = W
    

    sol = np.zeros((m,n))

    sol[0,:]  = [L_u(i) for i in x]
    sol[-1,:] = [L_i(i) for i in x]
    sol[:,0]  = [L_l(i) for i in y]
    sol[:,-1] = [L_r(i) for i in y]

    W = np.reshape(W, (m-2,n-2) )    

    sol[1:-1,1:-1] = W

    return sol, W1, x, y

#### - Los datos para el problema:

In [ ]:
Mat8, VecE, xE, yE = EDP_Eliptica_Mat(0, 0.5, 0, 0.5, 0.125, 0.125, lambda x, y: 0, lambda x: 0, lambda x: 200*x,lambda x: 0, lambda x: 200*x)


#### - Seleccionando los puntos de interseccion interiores del mallado: 

In [ ]:
n = len(xE)
m = len(yE)
Xp = []
Yp = []
for i in range(1,n-1):
    for j in range(1,m-1):
        Xp.append(xE[i])
        Yp.append(yE[j])
        

#### - Solucion Exacta: 
En este caso el error es 0, ya que la solucion exacta es $400xy$, que nos da $\displaystyle \frac{\partial^2 U}{\partial^2 x} (x,y) + \frac{\partial ^2 U}{\partial y^2} (x,y) = 0$

In [ ]:
def ExactaE(x,y):
    n = len(x)
    sol = []
    for i in range(n):
        sol.append(400*x[i]*y[i])
        
    return sol
asdj= ExactaE(Xp, Yp)

In [ ]:
dE = {'x':Xp, 'y':Yp, 'Aproximacion': VecE }
dE = pd.DataFrame(data=dE)
dE = dE[['x','y','Aproximacion']]
dE

#### - Ejemplo 2 (Burden - pag 722)
En este ejemplo, resolveremos la ecuacion de Poisson para $f(x,y) \neq 0$

$$\frac{\partial^2 U}{\partial^2 x} (x,t) + \frac{\partial ^2 U}{\partial y^2} (x,t) = xe^y \text{ , } 0 < x <2 \text{ , } 0 < y < 1$$

Con las condiciones frontera: 

$$U(0,y)= 0   \text{ , } U(2,y) = 2e^y \text{ , } 0 < y < 1$$
$$U(x,0)= x   \text{ , } U(x,1) = ex \text{ , } 0 < x < 2$$

#### - Los datos para el problema:

In [ ]:
Mat9, VecE2, xE2, yE2 = EDP_Eliptica_Mat(0,2,0,1,2/6,0.2,lambda x,y: x*np.exp(y),lambda x:0, lambda x: 2*np.exp(x),lambda x: x, lambda x:np.exp(1)*x)

In [ ]:
n = len(xE2)
m = len(yE2)
Xp2 = []
Yp2 = []
for i in range(1,n-1):
    for j in range(1,m-1):
        Xp2.append(xE2[i])
        Yp2.append(yE2[j])

#### - Solucion exacta: 
Para: $\displaystyle U(x,y) = x e^y$

In [ ]:
def ExactaE(x,y):
    n = len(x)
    sol = []
    for i in range(n):
        sol.append(x[i]*np.exp(y[i]))
        
    return sol
asdj2= ExactaE(Xp2, Yp2)

#### - Hallando el error: 

In [ ]:
ErrorE2 = ErrorAb(asdj2,VecE2)

In [ ]:
dE2 = {'x':Xp2, 'y':Yp2, 'Aproximacion': VecE2,'Error':ErrorE2,'Exacta':asdj2}
dE2 = pd.DataFrame(data=dE2)
dE2 = dE2[['x','y','Aproximacion','Exacta', 'Error']]
dE2

#### OJO: aca se nota un error muy grande ya que yo estoy usando el metodo matricial (el cual no es recomendado para mallados muy grandes) y en el libro usan el metodo iterativo.